In [1]:
from IPython import display
import requests
import json
from sodapy import Socrata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
import time
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_columns',50)

from sklearn.preprocessing import OneHotEncoder


# Introduction

According to the CDC, more than 48 million Americans per year become sick from food, and an estimated 75% of the outbreaks came from food prepared by caterers, delis, and restaurants. In most cities, health inspections are generally random, which can increase time spent on spot checks at clean restaurants that have been following the rules closely — and missed opportunities to improve health and hygiene at places with more pressing food safety issues.

The goal for this project is to leverage public citizen generated data from social media to narrow the search for critical health and safety violations in New York City. As the City of New York manages  an open data portal, everyone can access historical hygiene inspections and violation records. By combine these two data source this project aims to determine which words, phrases, ratings, and patterns among restaurants lead to critical health and safety violations. This model can assist city health inspectors do their job better by prioritizing the kitchens most likely to be in violation of code.

# Obtain

This project requires data pulled from two different sources, the City of New York and Yelp. To obtain the data we will call the API keys.

In [ ]:
# Las Vegas Data

In [2]:
inspections = pd.read_csv('data/Restaurant_Inspections_Open_Data.csv')
inspections

,ObjectId,Serial_Number,Permit_Number,Restaurant_Name,Location_Name,Category_Name,Address,City,State,Zip,Current_Demerits,Current_Grade,Date_Current,Inspection_Date,Inspection_Time,Employee_ID,Inspection_Type,Inspection_Demerits,Inspection_Grade,Permit_Status,Inspection_Result,Violations,Record_Updated,Location_1
0,1,DAPRX3DFB,PR0126049,POLLOS EL DORADO,POLLOS EL DORADO,Restaurant,5717 MERRILL,Las Vegas,Nevada,89120,0,A,2022-04-08 00:00:00,2022-04-08 00:00:00,2022-04-08 13:30:00,EE7000559,Re-inspection,0.0,A,NaN,A Grade,2930,2022-04-08 14:24:14,"(36.08527340, -115.07467880)"
1,2,DA7O2JSB1,PR0003157,7-ELEVEN #13703 - SNACK BAR,7-Eleven Food Store #13703,Snack Bar,5891 E Lake Mead Blvd,Las Vegas,Nevada,89156-6816,6,A,2022-04-08 00:00:00,2022-04-08 00:00:00,2022-04-08 07:00:00,EE7001675,Routine Inspection,6.0,A,NaN,A Grade,"215,2909,2930",2022-04-08 08:08:38,"(36.19587090, -115.04415700)"
2,3,DAYGUXFYM,PR0133375,JOE'S PLACE,JOE'S PLACE,Restaurant,375 GIBSON Rd,Henderson,Nevada,89014,8,A,2022-04-08 00:00:00,2022-04-08 00:00:00,2022-04-08 08:00:00,EE7001553,Routine Inspection,8.0,A,NaN,A Grade,"211,2954",2022-04-08 10:31:54,"(36.05439840, -115.02902200)"
3,4,DAYOQVLMW,PR0133375,JOE'S PLACE,JOE'S PLACE,Restaurant,375 GIBSON Rd,Henderson,Nevada,89014,8,A,2022-04-08 00:00:00,2022-04-08 00:00:00,2022-04-08 09:15:00,EE7001553,Routine Inspection,6.0,A,NaN,A Grade,"212,2912,2928,2930,2955",2022-04-08 10:48:30,"(36.05439840, -115.02902200)"
4,5,DANSRWNBW,PR0112829,FREDDY'S FROZEN CUSTARD & STEAKBURGERS,FREDDY'S FROZEN CUSTARD & STEAKBURGERS,Restaurant,1290 W WARM SPRINGS,Henderson,Nevada,89014,0,A,2022-04-08 00:00:00,2022-04-08 00:00:00,2022-04-08 10:30:00,EE7001809,Routine Inspection,0.0,A,NaN,A Grade,"2912,2930",2022-04-08 11:39:42,"(36.05681470, -115.04147690)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300595,300596,DA0503728,PR0013916,Chaser's Pub,Chaser's Pub,Bar / Tavern,6870 Spring Mountain Rd,Las Vegas,Nevada,89146-8856,6,A,2021-02-23 00:00:00,2005-01-04 00:00:00,2005-01-04 12:45:00,EE7000330,Routine Inspection,9.0,A,NaN,Compliant,"14,19,112",2013-02-21 22:26:12,"(36.12585700, -115.24223600)"
300596,300597,DA0503747,PR0013916,Chaser's Pub,Chaser's Pub,Bar / Tavern,6870 Spring Mountain Rd,Las Vegas,Nevada,89146-8856,6,A,2021-02-23 00:00:00,2005-01-04 00:00:00,2005-01-04 12:35:00,EE7000330,Routine Inspection,8.0,A,NaN,Compliant,"13,14,19",2013-02-21 22:26:12,"(36.12585700, -115.24223600)"
300597,300598,DA0124319,PR0003308,C J Market - Deli,C J Market,Snack Bar,1000 S Torrey Pines Dr,Las Vegas,Nevada,89107-3520,12,B,2011-09-08 00:00:00,2005-01-04 00:00:00,2005-01-04 15:30:00,EE7000619,Routine Inspection,0.0,A,NaN,Compliant,NaN,2013-02-21 22:26:12,"(36.15992900, -115.23496600)"
300598,300599,DA0499656,PR0013392,Agave Restaurant Front Line,Agave,Restaurant,10820 W Charleston Blvd,Las Vegas,Nevada,89135-1102,3,A,2012-06-20 00:00:00,2005-01-04 00:00:00,2005-01-04 12:55:00,EE7000373,Routine Inspection,4.0,A,NaN,Compliant,4,2013-02-21 22:26:12,"(36.16070900, -115.33139300)"


In [3]:
# Yelp dataset

In [19]:
businesses = pd.read_json('data/yelp_dataset/yelp_academic_dataset_business.json',lines=True)

In [ ]:
businesses.head(3)

In [15]:
NV = businesses[businesses['state'] == 'NV']

In [16]:
NV['city'].value_counts()

Reno               5932
Sparks             1623
Sun Valley           38
Virginia City        32
Verdi                29
Spanish Springs      25
Washoe Valley         4
New Washoe City       4
Cold Springs          2
Stead                 2
reno                  2
Carson City           2
Reno                  2
Reno City             1
Reno Nevada           1
Mount Laurel          1
Spark                 1
Nevada                1
sparks                1
McCarran              1
Fernley               1
Sparks,               1
South Reno            1
Sparks NV             1
Sparks                1
VC Highlands          1
SPARKS                1
RENO AP               1
Washoe                1
RENO                  1
Reno Sparks           1
Name: city, dtype: int64

In [21]:
with open('data/yelp_dataset/yelp_academic_dataset_review.json','r') as f:
    reviews = json.load(f)

JSONDecodeError: Extra data: line 2 column 1 (char 709)

## NYC Open Data API

In [ ]:
doh = pd.read_csv('data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
doh

In [ ]:
# How many unique restaurants are in this dataset?
doh['CAMIS'].nunique()

Health code violations found during an inspections carries a point value, and a restaurant’s score corresponds to a letter grade. A lower point score, leads to a better letter grade:

"A" grade: 0 to 13 points for sanitary violations
"B" grade: 14 to 27 points for sanitary violations
"C" grade: 28 or more points for sanitary violations

In [ ]:
doh['A'] = doh['SCORE'] < 14
doh['B'] = (doh['SCORE'] > 13) & (doh['SCORE'] < 28)
doh['C'] = doh['SCORE'] > 27

In [ ]:
doh

In [ ]:
pass_fail = doh.groupby(['CAMIS'])['PASS', 'FAIL'].sum()

In [ ]:
pass_fail[pass_fail['FAIL'] > 0]

Of the 19,792 unique restaurants, 12,221 did not pass the intitial inspections at least once.

In [ ]:
doh_grades = doh.groupby(['CAMIS'])['BORO','BUILDING','STREET','ZIPCODE',
                                    'PHONE','CUISINE DESCRIPTION',
                                    'Latitude','Longitude','Community Board',
                                    'Council District','Census Tract',
                                    'A', 'B','C'].sum()

In [ ]:
doh_grades

In [ ]:
doh_grades[doh_grades['B'] > 0]

Of the 19,792 unique restaurants, 9,978 failed an initial cycle inspection at least once.

In [ ]:
doh.duplicated().sum()

In [ ]:
doh.drop_duplicates(inplace=True)

In [ ]:
doh.shape

In [ ]:
doh['SCORE'].isna().sum()

In [ ]:
doh['SCORE'].hist(bins=113, figsize=(12,8));

In [ ]:
doh['SCORE'].value_counts()

In [ ]:
doh['SCORE'].mean()

In [ ]:
doh['SCORE'].median()

In [ ]:
doh['SCORE'].mode()

In [ ]:
doh['GRADE'].isna().sum()

In [ ]:
doh['GRADE'].value_counts()

In [ ]:
doh.info()

In [ ]:
doh['INSPECTION DATE'] =  pd.to_datetime(doh['INSPECTION DATE'])

In [ ]:
# doh.pivot(columns=['CAMIS','INSPECTION DATE'],values='SCORE')

The New York Health Department inspects the approximately 27,000 restaurants within the city to monitor their compliance with food safety regulations. Inspectors observe how food is prepared, served and stored and whether restaurant workers are practicing good hygiene. They check food temperatures, equipment maintenance and pest control measures.

Calling API

In [ ]:
# url = 'https://data.cityofnewyork.us/resource/43nn-pn8j.json?$limit=400000'
# nyc_response = requests.get(url)
# nyc_response.ok
# nyc_data = nyc_response.json()
# nyc_data
# pd.DataFrame(nyc_data)

In [ ]:
# dohmh_df = pd.read_csv('https://data.cityofnewyork.us/resource/43nn-pn8j.csv?$limit=400000')

In [ ]:
dohmh_df['score'].hist(bins='auto', figsize=(12,8));

In [ ]:
# # Take a look at duplicated records
# duplicates = df1.duplicated(subset=['camis'], keep=False)
# df1.loc[duplicates.loc[duplicates==True].index].sort_values(by='camis')

In [ ]:
# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata("data.cityofnewyork.us", None)



# # Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("43nn-pn8j", limit=2000)

# # Convert to pandas DataFrame
# dohmh_df = pd.DataFrame.from_records(results)

## Yelp API

In [ ]:
with open('/Users/Rob/.secret/yelp_api.json') as f:
    creds = json.load(f)

In [ ]:
creds.keys()

In [ ]:
#Business Search      
url = 'https://api.yelp.com/v3/businesses/search'

#Business Match       
# url = 'https://api.yelp.com/v3/businesses/matches'

#Phone Search         
# url = 'https://api.yelp.com/v3/businesses/search/phone'

#Business Details     
# url = 'https://api.yelp.com/v3/businesses/{id}'

#Business Reviews     
# url = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [ ]:
#Business Search  
headers = {
    'Authorization': 'Bearer ' + creds['api_key']
          }
location = 'New York NY'
SEARCH_LIMIT = 50

url_params = {
    'limit': SEARCH_LIMIT,
    'location' : location.replace(' ','+'),
    'offset': 0
             }


response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

In [ ]:
response_json = response.json()
# View the keys
response_json.keys()

In [ ]:
# Functionize the Yelp API call
def get_results(location, SEARCH_LIMIT=50,offset=0,
                url='https://api.yelp.com/v3/businesses/search',
                cred=None,fpath='/Users/Rob/.secret/yelp_api.json'):
    
    
    if cred is None:
    
        with open(fpath) as f:
            cred = json.load(f)

    headers = {
        'Authorization': 'Bearer ' + cred['api_key']
    }

    url_params = {
        'location' : location,
        'limit': SEARCH_LIMIT,
        'offset': offset
    }


    response = requests.get(url, headers=headers, params=url_params)
    return response.json()

In [ ]:
response = get_results('New York NY')

In [ ]:
response['total']

In [ ]:
total = response['total']

Yelp has information for 25,800 businesses in NYC.

In [ ]:
# Retrieve the value from response_json
businesses = response_json.get('businesses',{})
# View the first 2 records
businesses

In [ ]:
type(businesses)

In [ ]:
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
#         for key in ("name", "review_count", "rating", "price"):
#             prepared_data[key] = business_data.get(key, None)
    
#         # Parse and add latitude and longitude columns
#         coordinates = business_data["coordinates"]
#         prepared_data["latitude"] = coordinates["latitude"]
#         prepared_data["longitude"] = coordinates["longitude"]
        
#         # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
#     return results
    
# # Test out function
# prepared_businesses = prepare_data(businesses)
# prepared_businesses[:5]

### Loop Through Pagination

In [ ]:
def get_offsets(total):
    """
    Get a list of offsets needed to get all pages
    of data up until the total
    """
    return list(range(0, total, 50))

In [ ]:
url_params

In [ ]:
# full_dataset = []

# for offset in get_offsets(total):
    
#     url_params['offset'] = offset
    
#     response = requests.get(url, headers=headers, params=url_params)
    
#     time.sleep(1)
    
#     response_json = response.json()
    
#     businesses = response_json.get('businesses')
# #    
#     prepared_business = prepare_data(businesses)
    
#     full_dataset.extend(prepared_businesses)

# len(full_dataset)

In [ ]:
# Loop to retrieve all the results from the original request for all NYC businesses
offset=0
response = get_results('New York,NY',offset=offset)

n_per_page = len(response['businesses'])
total_result = response['total']

results = response_json.get('businesses',{})

while offset < total_result:
    offset+=n_per_page
    res = get_results('New York,NY',offset=offset)
    results.extend(res['businesses'])
results[:5]

In [ ]:
res

In [ ]:
len(results)

In [ ]:
df = pd.DataFrame(results)
df.to_csv('yelp_results2.csv',index=False)
df = pd.read_csv('yelp_results2.csv')
df

In [ ]:
# response.json().keys()

In [ ]:
# yelp_df = pd.DataFrame(response.json()['businesses'])
# # yelp_df.head(3)

In [ ]:
# lat = []
# long = []

# for _,business in yelp_df.iterrows():
#     lat.append(business['coordinates']['latitude'])
#     long.append(business['coordinates']['longitude'])

# yelp_df['lat'] = lat
# yelp_df['long'] = long

# Data Understanding

For this project there will be two sources and types of data used:

* Historical health and hygiene inspections recorded by New York City Department of Health and Mental Hygiene (DOHMH) public health inspectors
* User generated Yelp business ratings and reviews

## Understanding NYC DOHMH Data

This dataset contains over 330,000 records, let's explore its contents.

In [ ]:
dohmh_df['inspection_date'] =  pd.to_datetime(dohmh_inspections['inspection_date'])

In [ ]:
dohmh_df['inspection_date'].min()

In [ ]:
dohmh_df['inspection_date'].max()

Inspections in this dataset range from May 2008 up to present day.

In [ ]:
# dohmh_df.set_index('inspection_date')

In [ ]:
dohmh_df['boro'].value_counts()

In [ ]:
dohmh_df['score'].value_counts()

In [ ]:
dohmh_df['score'].isna().sum()

In [ ]:
dohmh_df['grade'].value_counts()

In [ ]:
dohmh_df['grade'].value_counts(normalize=True)

In [ ]:
dohmh_df['grade'].isna().sum()

In [ ]:
dohmh_df['critical_flag'].value_counts()

Critical violations are those most likely to contribute to foodborne illness

In [ ]:
dohmh_df['critical_flag'].value_counts(normalize=True)

In [ ]:
dohmh_df['inspection_type'].value_counts()

In [ ]:
dohmh_inspections = dohmh_df[(dohmh_df['inspection_type'] ==
                              'Cycle Inspection / Initial Inspection') | (dohmh_df['inspection_type'] ==
                              'Cycle Inspection / Re-inspection')]

In [ ]:
dohmh_inspections['critical_flag'].value_counts()

In [ ]:
# Drop records with NA critical_flag

dohmh_ohe = dohmh_inspections[dohmh_inspections['critical_flag'] != 'Not Applicable']

In [ ]:
dohmh_ohe = pd.get_dummies(dohmh_inspections,columns=['critical_flag'])

In [ ]:
dohmh_ohe.head(2)

In [ ]:
critical_flags = dohmh_ohe.groupby(['camis'])['critical_flag_Critical'].count()

In [ ]:
critical_flags[critical_flagss]

In [ ]:
critical_flags['critical_flag_Not Critical'].sum()

In [ ]:
non_critical_flags = dohmh_ohe.groupby(['camis'])

In [ ]:
dohmh_inspections.duplicated().sum()

In [ ]:
dohmh_inspections.loc[dohmh_inspections.duplicated(keep='first'),:]

In [ ]:
dohmh_inspections.drop_duplicates().shape

In [ ]:
dohmh_inspections.duplicated(subset=['camis']).sum()

In [ ]:
dohmh_inspections['camis'].nunique()

There are approximately 28,000 restaurants that have been inspected by the DOHMH. Let's check out how many have ever been flagged for a critical violation.

## Mapping with Follium